In [1]:
def buildWordVector(tokens, size, model):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens.split():
        try:       
            word = word.decode('utf-8')
            word_vec = model[word].reshape((1, size))             
            idf_weighted_vec = word_vec * tfidf_dict[word]
            vec += idf_weighted_vec
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [2]:
class IterableCorpus():
    
    def __init__(self, corpus):
        self.corpus = corpus
        
    def __iter__(self):
        for tweet in self.corpus:
            tweet_words = tweet.split()
            yield [word.decode('utf-8') for word in tweet_words]

In [3]:
import numpy as np
import gensim 
from gensim.models.word2vec import Word2Vec 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


from keras.wrappers.scikit_learn import KerasRegressor

from keras.models import Sequential
from keras.layers import Activation, Dense

from keras.layers import Dropout
from keras.constraints import maxnorm
from keras.optimizers import SGD

import numpy as np
import tensorflow as tf
import random as rn

import time

import helpers as HL
import cleaning as CL

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [4]:
#Global variables
N_DIMENTIONS = 200 

#subset? 
nr_pos_tweets = 100000
nr_neg_tweets = 100000
total_tweets = 200000

#full set?
#nr_pos_tweets = 1250000
#nr_neg_tweets = 1250000
#total_tweets = 2500000
test_set_tweets = 10000


In [5]:
corpus_filenames = ['train_pos.txt', 'train_neg.txt','test_data.txt'] 

full_corpus, training_file_lengths = HL.create_corpus(corpus_filenames)

#test_corpus, test_corpus_length = HL.create_corpus(['test_data.txt'])

print("Len training_corpus", len(full_corpus))
#print("Len test_corpus", len(test_corpus))

Len training_corpus 210000


In [6]:
cluster_file_path = '50mpaths2.txt'
cluster_dict = CL.create_dictionary(cluster_file_path)

print("Len cluster dict:", len(cluster_dict))

Len cluster dict: 216856


In [7]:
clusterised_full_corpus = CL.create_clusterized_corpus(full_corpus, cluster_dict)
#clusterised_test_corpus = CL.create_clusterized_corpus(test_corpus, cluster_dict)

print("Len clusterised training corpus: ", len(clusterised_full_corpus))
#print("Len clusterised test corpus: ", len(clusterised_test_corpus))

Len clusterised training corpus:  210000


In [8]:
#N-GRAMS

ngram_corpus = HL.creating_n_grams_cropus(corpus=clusterised_full_corpus, n_gram=2)

In [9]:
iterable_full_corpus = IterableCorpus(ngram_corpus)
#iterable_test_corpus = IterableCorpus(clusterised_test_corpus)

In [10]:
w2v_model = Word2Vec(iterable_full_corpus, size=N_DIMENTIONS, window=5, min_count=10, workers=4)
print("W2v test model:", w2v_model)

W2v test model: Word2Vec(vocab=44487, size=200, alpha=0.025)


In [11]:
w2v_model_wv = w2v_model.wv

del w2v_model

In [12]:
print(w2v_model_wv)

In [13]:
#Making labels
labels = np.zeros(total_tweets);
labels[0:nr_pos_tweets]=1;
labels[nr_pos_tweets:total_tweets]=0; 

print("labels shape: ", labels.shape)

labels shape:  (200000,)


In [14]:
vectorizer = TfidfVectorizer(
        min_df = 5, # removing word that occure less then 10 times 
        max_df = 0.5, # remove words that are too frequent ( more then 1.5 * number of tweets )
        sublinear_tf=True, # scale the term frequency in logarithmic scale
        use_idf =True
    )

In [20]:
corpus_tf_idf = vectorizer.fit_transform(ngram_corpus)

In [21]:
tfidf_dict = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [22]:
# SPLITTING THE W2V model into training and predict
train_clusterised_corpus = clusterised_full_corpus[:total_tweets:]
predict_clusterised_corpus = clusterised_full_corpus[total_tweets::]

print(len(train_clusterised_corpus))

200000


In [23]:
vecs_w2v = np.concatenate([buildWordVector(z, N_DIMENTIONS, w2v_model_wv) for z in train_clusterised_corpus])

In [24]:
training_vecs_w2v = scale(vecs_w2v)

In [25]:
test_vecs_w2v = np.concatenate([buildWordVector(z, N_DIMENTIONS, w2v_model_wv) for z in predict_clusterised_corpus])
test_vecs_w2v = scale(test_vecs_w2v)

In [80]:
print("Train w2v shape:",training_vecs_w2v.shape)
print("Train w2v shape:",test_vecs_w2v.shape)

Train w2v shape: (200000, 200)
Train w2v shape: (10000, 200)


In [16]:
#TING FOR Å SIKRE REPRODUSERBARHET ( ikke alt er nødv. nødvendig )

import os
os.environ['PYTHONHASHSEED'] = '0'
rn.seed(12345)

In [17]:
def basic_model():
    model = Sequential()
    model.add(Dense(100, input_dim=N_DIMENTIONS, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def basic_model_adam():
    model = Sequential()
    model.add(Dense(100, input_dim=N_DIMENTIONS, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def wide_model():
    model = Sequential()
    model.add(Dense(150, input_dim=N_DIMENTIONS, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def deep_1_model():
    model = Sequential()
    model.add(Dense(100, input_dim=N_DIMENTIONS, kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, input_dim=N_DIMENTIONS, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def deep_2_model():
    model = Sequential()
    model.add(Dense(100, input_dim=N_DIMENTIONS, kernel_initializer='normal', activation='relu'))
    model.add(Dense(60, input_dim=N_DIMENTIONS, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, input_dim=N_DIMENTIONS, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [27]:
seed = 7
np.random.seed(seed)

In [29]:
start = time.time()

kfold_basic_model = KFold(n_splits=10, random_state=seed)
estimator_basic_model = KerasRegressor(build_fn=basic_model, nb_epoch=10, batch_size=1024, verbose=1)

results_basic_model = cross_val_score(estimator_basic_model, training_vecs_w2v, labels, cv=kfold_basic_model)

print("Results: %.2f (%.2f) MSE" % (results_basic_model.mean(), results_basic_model.std()))
print("Time taken: ", (time.time() - start) / 60)

Epoch 1/10
180000/180000 [==============================] - 3s 16us/step - loss: 0.4883 - acc: 0.7474
Epoch 2/10
180000/180000 [==============================] - 2s 12us/step - loss: 0.4439 - acc: 0.7811
Epoch 3/10
180000/180000 [==============================] - 2s 12us/step - loss: 0.4298 - acc: 0.7904
Epoch 4/10
180000/180000 [==============================] - 2s 12us/step - loss: 0.4215 - acc: 0.7952
Epoch 5/10
180000/180000 [==============================] - 2s 12us/step - loss: 0.4152 - acc: 0.8004
Epoch 6/10
180000/180000 [==============================] - 2s 12us/step - loss: 0.4104 - acc: 0.8029
Epoch 7/10
180000/180000 [==============================] - 2s 12us/step - loss: 0.4066 - acc: 0.8045
Epoch 8/10
180000/180000 [==============================] - 2s 12us/step - loss: 0.4032 - acc: 0.8075
Epoch 9/10
180000/180000 [==============================] - 2s 12us/step - loss: 0.4004 - acc: 0.8088
Epoch 10/10
20000/20000 [==============================] - 0s 17us/step
Epoch 1/10

180000/180000 [==============================] - 2s 12us/step - loss: 0.4003 - acc: 0.8113
Epoch 8/10
180000/180000 [==============================] - 2s 13us/step - loss: 0.3969 - acc: 0.8132
Epoch 9/10
180000/180000 [==============================] - 2s 13us/step - loss: 0.3937 - acc: 0.8149
Epoch 10/10
20000/20000 [==============================] - 0s 22us/step
Epoch 1/10
180000/180000 [==============================] - 3s 19us/step - loss: 0.4851 - acc: 0.7549
Epoch 2/10
180000/180000 [==============================] - 3s 14us/step - loss: 0.4380 - acc: 0.7883
Epoch 3/10
180000/180000 [==============================] - 2s 13us/step - loss: 0.4240 - acc: 0.7969
Epoch 4/10
180000/180000 [==============================] - 2s 14us/step - loss: 0.4157 - acc: 0.8022
Epoch 5/10
180000/180000 [==============================] - 2s 14us/step - loss: 0.4093 - acc: 0.8056
Epoch 6/10
180000/180000 [==============================] - 2s 13us/step - loss: 0.4047 - acc: 0.8079
Epoch 7/10
180000/180

In [30]:
print(results_basic_model)

[-0.42963775 -0.51358442 -0.47425266 -0.47768632 -0.47643836 -0.50494956
 -0.52415656 -0.49545222 -0.5434581  -0.51551282]


In [ ]:
estimator_basic_model_adam = KerasRegressor(build_fn=basic_model_adam, nb_epoch=5, batch_size=1024, verbose=1)
kfold_basic_model_adam = KFold(n_splits=5, random_state=seed)

results_basic_model_adam = cross_val_predict(estimator_basic_model_adam, training_vecs_w2v, labels, cv=kfold_basic_model_adam)
print("Results: %.2f (%.2f) MSE" % (results_basic_model_adam.mean(), results_basic_model_adam.std()))

In [ ]:
estimator_model_wide = KerasRegressor(build_fn=wide_model, nb_epoch=5, batch_size=1024, verbose=1)
kfold_basic_model = KFold(n_splits=5, random_state=seed)

results_wide_model = cross_val_predict(estimator_model_wide, test_vecs_w2v, labels, cv=kfold_basic_model)
print("Results: %.2f (%.2f) MSE" % (results_wide_model.mean(), results_wide_model.std()))



In [85]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

early_stopping = EarlyStopping(monitor='loss', patience=3)
checkpointer = ModelCheckpoint(filepath='tmp/saved_weights.hdf5', verbose=1, save_best_only=True)

model = wide_model()

model.fit(training_vecs_w2v, labels, epochs=30, batch_size=1024, verbose=1, callbacks=[early_stopping, checkpointer], validation_split=0.15)


Train on 170000 samples, validate on 30000 samples
Epoch 1/30
170000/170000 [==============================] - 3s 20us/step - loss: 0.4838 - acc: 0.7571 - val_loss: 0.5897 - val_acc: 0.6604
Epoch 2/30
170000/170000 [==============================] - 2s 14us/step - loss: 0.4301 - acc: 0.7951 - val_loss: 0.5614 - val_acc: 0.6871
Epoch 3/30
170000/170000 [==============================] - 3s 15us/step - loss: 0.4147 - acc: 0.8042 - val_loss: 0.5698 - val_acc: 0.6855
Epoch 4/30
170000/170000 [==============================] - 3s 15us/step - loss: 0.4047 - acc: 0.8100 - val_loss: 0.5538 - val_acc: 0.6984
Epoch 5/30
170000/170000 [==============================] - 2s 15us/step - loss: 0.3993 - acc: 0.8134 - val_loss: 0.5214 - val_acc: 0.7230
Epoch 6/30
170000/170000 [==============================] - 2s 14us/step - loss: 0.3956 - acc: 0.8153 - val_loss: 0.5877 - val_acc: 0.6809
Epoch 7/30
170000/170000 [==============================] - 2s 14us/step - loss: 0.3887 - acc: 0.8196 - val_loss: 0

In [86]:
preds = model.predict(test_vecs_w2v)

In [87]:
preds

rounded = [-1 if round(x[0])==0 else 1 for x in preds]

print(rounded)

[-1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -

In [88]:
ids = list(range(1,test_set_tweets+1))
y_pred = rounded
name = "keggle_submission_neural.csv"

HL.create_csv_submission(ids, y_pred, name)

In [ ]:
#KAN BRUKES TIL NÅR MAN SKAL GJØRE EN MODEL FIT, NÅR MAN HAR FUNNET RIKTIG MODELL Å BRUKE OVER

# Smaller batch equalls longer run times, but less epochs for convergence 
# DENNE GA 80% PÅ KAGGLE

#from keras.callbacks import EarlyStopping
#from keras.callbacks import ModelCheckpoint

#early_stopping = EarlyStopping(monitor='loss', patience=2)
#checkpointer = ModelCheckpoint(filepath='tmp/saved_weights.hdf5', verbose=1, save_best_only=True)

#model_4 = Sequential()
#model_4.add(Dropout(0.05, input_shape=(N_DIMENTIONS,)))
#model_4.add(Dense(100, activation='relu', input_dim=N_DIMENTIONS))
#model_4.add(Dense(60, activation='relu'))
#model_4.add(Dense(30, activation='relu'))
#model_4.add(Dropout(0, input_shape=(N_DIMENTIONS,)))
#model_4.add(Dense(1, activation='sigmoid'))

#sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)

#model_4.compile(optimizer='rmsprop',
#      loss='binary_crossentropy',
#      metrics=['accuracy'])

#model_4.fit(training_vecs_w2v, labels, epochs=30, batch_size=1024, verbose=1, callbacks=[early_stopping, checkpointer], validation_split=0.15)

In [ ]:
from keras.models import load_model

best_model = load_model('tmp/saved_weights.hdf5')

In [ ]:
predictions = best_model.predict(test_vecs_w2v)

print(predictions)

rounded = [-1 if round(x[0])==0 else 1 for x in predictions]

print(rounded)

In [ ]:
ids = list(range(1,test_set_tweets+1))
y_pred = rounded
name = "keggle_submission_neural.csv"

HL.create_csv_submission(ids, y_pred, name)